<a href="https://colab.research.google.com/github/hong126-ch/CIS5450/blob/main/8_Module_2_Part_IV_Parallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture Module 2 Part 4: Parallelism

## LinkedIn Social Analysis

We now consider extensions of processing to multi-core and multi-machine models.  This includes:

* Polars, a multi-core Pandas-style library that is seeing wide adoption due to its performance (and flexible programming model)
* Sharded execution, a technique that is the basis of Spark's execution model.


In [1]:
!wget -nc https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

--2025-10-06 09:44:49--  https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.180.207, 172.253.115.207, 172.253.122.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.180.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179851696 (172M) [application/octet-stream]
Saving to: ‘linkedin_anon.jsonl’

linkedin_anon.jsonl 100%[===================>] 171.52M   216MB/s    in 0.8s    

2025-10-06 09:44:50 (216 MB/s) - ‘linkedin_anon.jsonl’ saved [179851696/179851696]



## PennGrader Setup

In [2]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [3]:
!pip3 install penngrader-client

In [4]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 64660501 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [5]:
%set_env HW_ID=cis5450_25f_HW9

env: HW_ID=cis5450_25f_HW9


In [6]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 64660501

Make sure this correct or we will not be able to store your grade


# Loading our Data

Now that we've seen how to do fairly complex queries over data in relations, we'll "pop back" to our big data example, which is the LinkedIn dataset.  Recall that we had a segment of the LinkedIn input file in our previous examples earlier in this module.

In [7]:
import pandas as pd
import numpy as np

# JSON parsing
import json

# Time conversions
import time

In [8]:
'''
Simple code to pull out data from JSON and load into DuckDB.
'''
linked_in = open('linkedin_anon.jsonl')
people_df = pd.read_json('linkedin_anon.jsonl', lines=True)

In [9]:
people_df

,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
1,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
2,tart-acorn,"{'family_name': 'Hannay', 'given_name': 'Passe...","San Francisco, California","[DNA, Nanotechnology, Molecular Biology, Softw...",Research,I am interested in inventing new methods to co...,https://www.linkedin.com/in/tart-acorn,"[{'major': 'Biophysics', 'end': '2009', 'name'...",None,0.0,"[{'org': 'UCSF', 'title': 'Assistant Professor...",None,[{'from': 'Wyss Institute for Biologically Ins...,"personal genomics, nanotechnology",None
3,objective-riesling,"{'family_name': 'Carnegie', 'given_name': 'Pas...",San Francisco Bay Area,None,Information Technology and Services,OBJECTIVE<Primary> Work on an interesting and ...,https://www.linkedin.com/in/objective-riesling,None,"{'affilition': ['Big Data, Low Latency', 'Expe...",5.0,"[{'org': '<Online Recruiting Company>', 'desc'...",None,"[{'from': '<Employee Benefits, Administration ...",None,None
4,generative-amberjack,"{'family_name': 'Duncan', 'given_name': 'Merri...","Chennai Area, India","[Program Management, French, Avionics, Embedde...",Aviation & Aerospace,"Experience in Avionics Systems, Embedded Syste...",https://www.linkedin.com/in/generative-amberjack,"[{'start': '1988', 'end': '1989', 'name': 'Eco...",{'member': 'Member of Project Management Insti...,NaN,None,None,None,"Literature, Philosophy, Music",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,glowing-flush,"{'family_name': 'Kincaid', 'given_name': 'Pass...",Greater Chicago Area,"[Spanish-speaking, Cicerone-Certified Beer Ser...",Marketing and Advertising,Sales and marketing professional specializing ...,https://www.linkedin.com/in/glowing-flush,"[{'start': '2007', 'major': 'PR and Advertisin...",None,5.0,"[{'org': 'Louis Glunz Beer Inc.', 'title': 'On...",None,"[{'from': 'Peet's Coffee and Tea', 'to': 'Expl...","craft beer industry, coffee industry, running,...",None
49996,grouchy-flight,"{'family_name': 'Ogilvy', 'given_name': 'Bulli...",Greater Atlanta Area,None,Financial Services,Accomplished business development manager expe...,https://www.linkedin.com/in/grouchy-flight,"[{'major': 'Marketing Focus', 'end': '2008', '...","{'member': 'Sigma Chi Fraternity', 'affilition...",14.0,"[{'org': 'Georgia-Pacific LLC', 'title': 'Acco...",None,"[{'from': 'Bayer Advanced', 'to': 'BBDO', 'tit...",None,[National Deans List]
49997,dense-bell,"{'family_name': 'Macdougall', 'given_name': 'B...","Calgary, Canada Area","[Project Management, Electrical Engineering, M...",Design,Brad Gibson is a recognized expert in power qu...,https://www.linkedin.com/in/dense-bell,[{'major': 'Engineering Physics (Solid State E...,"{'member': 'IEEE, APEGGA, APEGBC, PEO, APEGS, ...",42.0,"[{'org': 'DIALOG', 'desc': 'Electrical Enginee...","Data center design, high reliability power, po...","[{'from': 'Current Thinking Inc.', 'to': 'The ...",None,None
49998,brave-hoops,"{'family_name': 'Forsyth', 'given_name': 'Cadb...",San Francisco Bay Area,"[Corporate Social Responsibility, Public Polic...",Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,"[{'major': 'Law', 'end':

In [10]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)

# Take the lists, drop any blank strings
specialties_df = people_df[['_id','specilities']].explode('specilities')
specialties_df.dropna(inplace=True)
interests_df = people_df[['_id','interests']].explode('interests')
interests_df.dropna(inplace=True)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

education_df = get_nested_list_dict(people_df[['_id','education']], 'education')
experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')
skills_df = get_nested_list(people_df[['_id','skills']], 'skills')
honors_df = get_nested_list(people_df[['_id','honors']], 'honors')
events_df = get_nested_list_dict(people_df[['_id','events']], 'events')

groups_df = get_nested_dict(people_df[['_id','group']], 'group')

people_only_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests']).merge(names_df,on='_id')

# Multi-Core Execution of Dataframes

## Polars

Polars is a "drop-in replacement" for Pandas that supports more advanced processing, including multicore processing.

Its syntax is not exactly the same as Pandas', but close.  (In fact, it has some similarities to Apache Spark DataFrames, which we'll be seeing shortly.)

### Polars DataFrames

To take advantage of Polars you want to convert your dataframes from Pandas to Polars. (If you have base files, you can also use Polars' read_csv etc as you would from Pandas.)

In [11]:
import polars as pl

# Convert all Pandas dataframes to Polars Dataframes
people_only_pdf = pl.from_pandas(people_only_df)
education_pdf = pl.from_pandas(education_df)
experience_pdf = pl.from_pandas(experience_df)
skills_pdf = pl.from_pandas(skills_df)
groups_pdf = pl.from_pandas(groups_df)
names_pdf = pl.from_pandas(names_df)

## PL.Col

In Polars, when you are referring to a column in a DataFrame expression, this is represented by a *column object*.  For instance, you can express predicates on the values in a column when filtering, or you can request that certain columns be projected.

You can, e.g., talk about `pl.col('_id')`.  SQL-style, you can also refer to `pl.col('*')` if you are asking for all columns.

## Selection and Projection in Polars

Rather than using Pandas' `x[x[condition]]` notation, Polars has a *filter* function.   To project, you use a function called `select` (this is a bit confusing, but think SQL SELECT, not relational algebra select).

In [12]:
people_only_pdf.\
  select(pl.col('_id','family_name','given_name')).\
  filter(pl.col('family_name') <= 'Smith')

_id,family_name,given_name
str,str,str
"""moist-vodka""","""Post""","""Belvedere"""
"""tart-acorn""","""Hannay""","""Passepartout"""
"""objective-riesling""","""Carnegie""","""Passepartout"""
"""generative-amberjack""","""Duncan""","""Merriman"""
"""chalky-tenement""","""Graham""","""Leporello"""
…,…,…
"""glowing-flush""","""Kincaid""","""Passepartout"""
"""grouchy-flight""","""Ogilvy""","""Bullimore"""
"""dense-bell""","""Macdougall""","""Barrymore"""


## Joins

In many settings, multi-way joins are the most expensive query operations -- partly because they "blow up the data". You saw this with our discussions about join ordering.

We should be able to see the benefits of Polars multicore processing with some join expressions. Note that we still need to keep everything in memory for Pandas/Polars, and in fact there are simple join queries (say, all people joined with experiences and education) that will cause us to run out of memory.

Let's try a simpler query that joins people with some of their other info.

### Pandas Version

In [13]:
%%time
people_only_df.merge(skills_df, on='_id').merge(education_df, on='_id').merge(groups_df, on='_id')

CPU times: user 1 s, sys: 174 ms, total: 1.18 s
Wall time: 1.21 s


,_id,locality,industry,summary,url,interval,family_name,given_name,skills,start,major,end,name,desc,degree,affilition,member
0,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,2008,Economics,2008,Columbia University - Columbia Business School,"Coursework ""Principals of Economics"" ECON1105\...",NaN,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
1,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,2007,NaN,2007,Columbia University - Columbia Business School,NaN,NaN,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
2,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,1996,Cancer genomics,2001,Göteborgs universitet,"Thesis: ""The role of p53 in tumor progression ...",Ph.D.,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
3,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,1994,"Biology, Medicine;German Language",1995,Universität Regensburg,NaN,"Cancer Research, Coursework","[Big Data, Low Latency, Experts Answer's, Link...",NaN
4,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,1989,Biology,1994,Göteborgs universitet,,Master,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368157,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,1978,International Relations,1981,"University of California, Davis",,BA,NaN,NaN
368158,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,NaN,NaN,NaN,Great Zimbabwe University,,NaN,NaN,NaN
368159,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,1981,Law,1984,"University of California, Hastings College of ...",,Juris Doctor,NaN,NaN
368160,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,1978,International Relations,1981,"University of California, Davis",,BA,NaN,NaN


### Polars Version

Notice this finishes in roughly half the time as Pandas!

In [14]:
%%time
print(people_only_pdf.join(skills_pdf, on='_id').join(education_pdf, on='_id').join(groups_pdf, on='_id'))

shape: (368_162, 17)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬────────┐
│ _id        ┆ locality   ┆ industry  ┆ summary   ┆ … ┆ desc      ┆ degree    ┆ affilitio ┆ member │
│ ---        ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ n         ┆ ---    │
│ str        ┆ str        ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ ---       ┆ str    │
│            ┆            ┆           ┆           ┆   ┆           ┆           ┆ list[str] ┆        │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪════════╡
│ adagio-cat ┆ Antwerp    ┆ Pharmaceu ┆ Ph.D.     ┆ … ┆ Coursewor ┆ null      ┆ ["Big     ┆ null   │
│ alyst      ┆ Area,      ┆ ticals    ┆ scientist ┆   ┆ k "Princi ┆           ┆ Data, Low ┆        │
│            ┆ Belgium    ┆           ┆ with back ┆   ┆ pals of   ┆           ┆ Latency", ┆        │
│            ┆            ┆           ┆ groun…    ┆   ┆ Econ…     ┆   

## Laziness

In [15]:
%%time
result_pdf = people_only_pdf.join(skills_pdf, on='_id').\
  join(education_pdf, on='_id').join(groups_pdf, on='_id')
only_sf_pdf = result_pdf.filter(pl.col('locality') == 'San Francisco Bay Area')
print (len(only_sf_pdf))

10712
CPU times: user 213 ms, sys: 32.9 ms, total: 246 ms
Wall time: 167 ms


In [16]:
%%time
result_pdf = people_only_pdf.lazy().join(skills_pdf.lazy(), on='_id').join(education_pdf.lazy(), on='_id').join(groups_pdf.lazy(), on='_id').lazy()
only_sf_pdf = result_pdf.filter(pl.col('locality') == 'San Francisco Bay Area')
print (len(only_sf_pdf.collect()))

10712
CPU times: user 25.8 ms, sys: 2.79 ms, total: 28.6 ms
Wall time: 19.5 ms


In [17]:
results = only_sf_pdf.explain()

for line in results.split('\n'):
  print(line)

INNER JOIN:
LEFT PLAN ON: [col("_id")]
  INNER JOIN:
  LEFT PLAN ON: [col("_id")]
    INNER JOIN:
    LEFT PLAN ON: [col("_id")]
      FILTER [(col("locality")) == (String(San Francisco Bay Area))] FROM
        DF ["_id", "locality", "industry", "summary", ...]; PROJECT */8 COLUMNS
    RIGHT PLAN ON: [col("_id")]
      DF ["_id", "skills"]; PROJECT */2 COLUMNS
    END INNER JOIN
  RIGHT PLAN ON: [col("_id")]
    DF ["_id", "start", "major", "end", ...]; PROJECT */7 COLUMNS
  END INNER JOIN
RIGHT PLAN ON: [col("_id")]
  DF ["_id", "affilition", "member"]; PROJECT */3 COLUMNS
END INNER JOIN


# Understanding Sharded Execution

Let's now look (in simulation) at how sharding works!  This is the basis of cluster-based processing.

## A Data Graph in Relations

To do this we'll adapt our LinkedIn data to a *graph* scenario, with people and companies.

In [18]:
persons_df = people_only_df[['_id','family_name','given_name']]

persons_df

,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
1,adagio-catalyst,Watt,Brunton
2,tart-acorn,Hannay,Passepartout
3,objective-riesling,Carnegie,Passepartout
4,generative-amberjack,Duncan,Merriman
...,...,...,...
51817,glowing-flush,Kincaid,Passepartout
51818,grouchy-flight,Ogilvy,Bullimore
51819,dense-bell,Macdougall,Barrymore
51820,brave-hoops,Forsyth,Cadbury


In [19]:
experience_df

,_id,org,title,end,start,desc
0,adagio-catalyst,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers",Present,November 2009,Biomarker Leader for compounds in clinical dev...
1,adagio-catalyst,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics,NaN,September 2008,Single Cell Gene expression.
2,adagio-catalyst,Columbia University,Associate Research Scientist,NaN,August 2006,Work on peptide to restore wt p53 function in ...
3,adagio-catalyst,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow,NaN,January 2003,Molecular profiling of colorectal cancer.
4,adagio-catalyst,Sahlgrenska University Hospital,Research Scientist,NaN,November 2001,Cancer Research at Dept of Surgery.Molecular p...
...,...,...,...,...,...,...
1761792,fixed-partition,Four Seasons Hotels and Resorts,Communications Assistant,NaN,1995,NaN
1761793,fixed-partition,M Booth & Associates,"Senior Vice President, Director",Present,March 2002,"Director, Consumer Tech & Travel"
1761794,fixed-partition,Hill & Knowlton,Account Supervisor,NaN,September 1999,NaN
1761795,fixed-partition,Richmond Public Relations,Account Executive,NaN,1996,NaN


Companies are also nodes.  We'll auto-assign IDs for these.

In [20]:
companies_df = experience_df[['org']].drop_duplicates()
companies_df['company_id'] = companies_df.index
companies_df

,org,company_id
0,Johnson and Johnson,0
1,Albert Einstein Medical Center,1
2,Columbia University,2
3,Memorial Sloan Kettering Cancer Center,3
4,Sahlgrenska University Hospital,4
...,...,...
1761646,President's Task Force on Health Care Reform,1761646
1761647,"Energy and Commerce Subcommittee on Commerce, ...",1761647
1761648,Congresswoman Cardiss Collins,1761648
1761781,M Booth & Associates,1761781


Finally, here are some edges.  We preserve the relationships between people and their experiences with organizations.

In [21]:
worked_for = experience_df[['_id', 'org', 'title']].copy().merge(companies_df.set_index('org'), on='org').drop_duplicates()
worked_for = worked_for.drop(columns=['org'])
worked_for

,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Associate Research Scientist,2
3,adagio-catalyst,Post Doctoral Research Fellow,3
4,adagio-catalyst,Research Scientist,4
...,...,...,...
1761648,brave-hoops,Legislative Counsel: U.S. House of Representat...,1761648
1761781,fixed-partition,"Senior Vice President, Director",1761781
1761782,fixed-partition,Account Supervisor,146464
1761783,fixed-partition,Account Executive,1761783


### Data

In [22]:
print('persons_df:')
display(persons_df)
print('companies_df:')
display(companies_df)
print('worked_df:')
display(worked_for)

persons_df:


,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
1,adagio-catalyst,Watt,Brunton
2,tart-acorn,Hannay,Passepartout
3,objective-riesling,Carnegie,Passepartout
4,generative-amberjack,Duncan,Merriman
...,...,...,...
51817,glowing-flush,Kincaid,Passepartout
51818,grouchy-flight,Ogilvy,Bullimore
51819,dense-bell,Macdougall,Barrymore
51820,brave-hoops,Forsyth,Cadbury


companies_df:


,org,company_id
0,Johnson and Johnson,0
1,Albert Einstein Medical Center,1
2,Columbia University,2
3,Memorial Sloan Kettering Cancer Center,3
4,Sahlgrenska University Hospital,4
...,...,...
1761646,President's Task Force on Health Care Reform,1761646
1761647,"Energy and Commerce Subcommittee on Commerce, ...",1761647
1761648,Congresswoman Cardiss Collins,1761648
1761781,M Booth & Associates,1761781


worked_df:


,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Associate Research Scientist,2
3,adagio-catalyst,Post Doctoral Research Fellow,3
4,adagio-catalyst,Research Scientist,4
...,...,...,...
1761648,brave-hoops,Legislative Counsel: U.S. House of Representat...,1761648
1761781,fixed-partition,"Senior Vice President, Director",1761781
1761782,fixed-partition,Account Supervisor,146464
1761783,fixed-partition,Account Executive,1761783


### Implementing Sharded Computations - Our Operators

Rather than look at sharding in Spark, we'll instead simulate having 4 machines, each of which has access to a shard of each table.

To illustrate the basic concepts we'll create our own basic implementations of Relational Algebra over sharded data -- as well as our own `repartition` etc.

In [23]:
nodes = 4

def get_hash(x):
  '''
  Hash code for data, used to compute a shard
  '''
  if isinstance(x, int):
    return x
  elif isinstance(x, str):
    return x.__hash__()
  elif pd.isna(x):
    return 'null'.__hash__()
  else:
    raise RuntimeError('Cannot hash {}'.format(type(x)))

def get_shard(sharded_df: list[pd.DataFrame], index: int):
  '''
  Retrieves the sub-dataframe corresponding to a shard
  '''
  return sharded_df[index]

def repartition(df_or_shards, key: str, shards: int = nodes):
  '''
  Take a dataframe or a list of sharded sub-dataframes.
  Repartition them on the key, based on the number of nodes.
  Return the new list of sub-dataframes.
  '''
  data = df_or_shards
  if isinstance(df_or_shards, list):
    data = pd.concat(df_or_shards)

  resharded_df = []
  for shard in range(0, shards):
    resharded_df.append(data[data[key].apply(lambda x: get_hash(x) % shards == shard)])

  return resharded_df

def rename_shards(sharded_df: list[pd.DataFrame], renamer: dict):
  '''
  Simple relational algebra operator:
  Rename the columns of a list of sharded dataframes
  '''
  renamed_df = []
  for shard in range(0, len(sharded_df)):
    renamed_df.append(sharded_df[shard].rename(columns=renamer))

  return renamed_df

def filter_shards(sharded_df: list[pd.DataFrame], filter_fn):
  '''
  Simple relational algebra operator:
  Filter a list of sharded dataframes with a predicate
  '''
  filtered_df = []
  for shard in range(0, len(sharded_df)):
    filtered_df.append(
        sharded_df[shard][
            sharded_df[shard].apply(filter_fn, axis=1)])

  return filtered_df


def join_shards(sharded_df1: list[pd.DataFrame], sharded_df2: list[pd.DataFrame],
                left_key, right_key):
  '''
  Simple relational algebra operator:
  Join two lists of sharded dataframes
  '''
  ret_result = []

  for shard in range(0, len(sharded_df1)):
    ret_result.append(sharded_df1[shard].merge(sharded_df2[shard], left_on=left_key, right_on=right_key))

  return ret_result

def join_shard_with_broadcast(sharded_df1: list[pd.DataFrame], sharded_df2: pd.DataFrame,
                left_key, right_key):
  '''
  Simple relational algebra operator:
  Join a list of sharded dataframes with a single dataframe
  that conceptually gets "broadcast" to all of the shards
  '''
  ret_result = []

  for shard in range(0, len(sharded_df1)):
    ret_result.append(sharded_df1[shard].merge(sharded_df2, left_on=left_key, right_on=right_key))

  return ret_result

## An Example Query

Suppose we simply want to ask this query:

`(persons_df) -[worked_for]--> (companies_df)`

## Shard the Relations!

We'll create versions of this:
* `sharded_persons` (by _id)
* `sharded_companies` (by company)
* Two shards of `worked_for`:
  - `sharded_by_user`
  - `sharded_by_company`

In [24]:
# Generally, for a table like persons, we would
# shard by default on the key (_id).  We can
# see each shard here.

sharded_persons = repartition(persons_df, '_id')

for p in sharded_persons:
  display(p)

,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
1,adagio-catalyst,Watt,Brunton
3,objective-riesling,Carnegie,Passepartout
6,chalky-tenement,Graham,Leporello
16,plain-torpedo,Hamilton,Brunton
...,...,...,...
51803,orthogonal-liqueur,Maclean,Cadbury
51813,joint-atoll,Macalister,Cadbury
51817,glowing-flush,Kincaid,Passepartout
51818,grouchy-flight,Ogilvy,Bullimore


,_id,family_name,given_name
2,tart-acorn,Hannay,Passepartout
4,generative-amberjack,Duncan,Merriman
5,salty-section,Watt,Figaro
8,arctic-investor,Rollo,Merriman
9,rich-laser,Spalding,Figaro
...,...,...,...
51807,sleek-sopapillas,Wheatley,Barrymore
51808,resolving-image,Agnew,Jenkins
51816,tense-spline,Cunningham,Merriman
51820,brave-hoops,Forsyth,Cadbury


,_id,family_name,given_name
11,afraid-collision,Hunter,Merriman
12,oily-sport,Burnett,Merriman
13,forgiving-desert,Haig,Belvedere
15,medium-marlin,Primrose,Belvedere
24,fancy-notch,Macmillan,Barrymore
...,...,...,...
51805,radial-ribbon,Sinclair,Merriman
51806,direct-scenario,Boyd,Brunton
51809,legato-rook,Keen,Belvedere
51810,late-cadet,Mcmullen,Jeeves


,_id,family_name,given_name
7,sensitive-estuary,Macneil,Merriman
17,cheerful-broker,Barclay,Cadbury
18,coped-wash,Buchanan,Figaro
19,hard-node,Allen,Brunton
22,tart-velocity,Sutherland,Poole
...,...,...,...
51802,pizzicato-burst,Lough,Jeeves
51804,espressivo-media,Dundas,Brunton
51812,flat-leading,Lewis,Cadbury
51814,natural-customer,Charteris,Passepartout


In [25]:
# Similarly with companies... we shard by company_id
sharded_companies = repartition(companies_df, 'company_id')

for c in sharded_companies:
  display(c)

,org,company_id
0,Johnson and Johnson,0
4,Sahlgrenska University Hospital,4
32,<Software Consulting Company>,32
40,<Employment Agency>,40
44,<Biotech>,44
...,...,...
1761492,Goose Island Beer Company,1761492
1761576,WestWayne,1761576
1761624,The EA Group,1761624
1761644,CA Manufacturing Technology Center,1761644


,org,company_id
1,Albert Einstein Medical Center,1
25,UCSF,25
29,<Online Recruiting Company>,29
37,<Psychology Organization>,37
41,<Biometrics Hardware Company>,41
...,...,...
1761621,DIALOG,1761621
1761637,The Bipartisan Bridge,1761637
1761641,Nielsen Media's Independent Task Force on Tele...,1761641
1761645,California State Controller’s Office,1761645


,org,company_id
2,Columbia University,2
26,Wyss Institute for Biologically Inspired Engin...,26
30,<Medical Testing Company>,30
34,<Temperature Sensor Manufacturer>,34
42,<Product Manufacturing Company>,42
...,...,...
1761574,Celulosa Arauco,1761574
1761578,Bayer Advanced,1761578
1761638,International Commission on Workforce Development,1761638
1761642,"State of California Business, Transportation, ...",1761642


,org,company_id
3,Memorial Sloan Kettering Cancer Center,3
31,000Montgomery.Com,31
35,<Advertising Company>,35
39,<National Recruitment Network>,39
43,<Emergency Services>,43
...,...,...
1761575,"House Parts, Inc.",1761575
1761623,Current Thinking Inc.,1761623
1761639,"United Nations, Global Alliance on ICT (inform...",1761639
1761647,"Energy and Commerce Subcommittee on Commerce, ...",1761647


Edges are inherently trickier to shard: we can shard by the *source* or by the *destination* but can't simultaneously do so for a single table, by both.  We can, of course, create two dataframes - one sharded by each.

In [26]:
# Edge sharded by user ID
sharded_by_user = repartition(worked_for, '_id')

for edge in sharded_by_user:
  display(edge)

,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Associate Research Scientist,2
3,adagio-catalyst,Post Doctoral Research Fellow,3
4,adagio-catalyst,Research Scientist,4
...,...,...,...
1761578,grouchy-flight,Intern,1761578
1761621,dense-bell,Principal,1761621
1761622,dense-bell,Project Manager,913976
1761623,dense-bell,Managing Partner,1761623


,_id,title,company_id
25,tart-acorn,Assistant Professor,25
26,tart-acorn,Technology Development Fellow,26
870,salty-section,Founding Partner and Client Services Director,870
871,salty-section,Managing Partner,871
872,salty-section,Senior Account Director,872
...,...,...,...
1761648,brave-hoops,Legislative Counsel: U.S. House of Representat...,1761648
1761781,fixed-partition,"Senior Vice President, Director",1761781
1761782,fixed-partition,Account Supervisor,146464
1761783,fixed-partition,Account Executive,1761783


,_id,title,company_id
1080,afraid-collision,Zaakvoerster,1080
1081,afraid-collision,HR,1081
1082,afraid-collision,Branch Manager Kapellen at Adecco - Maritime/I...,1082
1083,afraid-collision,Branch Manager,1082
1084,afraid-collision,Branch Manager Schoten - Lier,1084
...,...,...,...
1761101,direct-scenario,Summer Associate,327514
1761102,direct-scenario,Investment Banking Analyst,32842
1761170,legato-rook,Member of the Technical Staff,5151
1761171,legato-rook,Chief Architect,1761171


,_id,title,company_id
979,sensitive-estuary,Sales Manager - Security Team,979
980,sensitive-estuary,Information Security Consultant,979
981,sensitive-estuary,IT Security Sales Consultant,981
982,sensitive-estuary,Sales Manager,982
983,sensitive-estuary,Corporate Account Manager,983
...,...,...,...
1761457,resultant-coach,"Director - Head of CDO Management, Asset Backe...",289179
1761458,resultant-coach,"Chairman, Board of Directors",1761458
1761459,resultant-coach,Associate,1761459
1761460,resultant-coach,Associate,1761460


Here's the other version...  Sharded by the company ID.

In [27]:
sharded_by_companies = repartition(worked_for, 'company_id')

for edge in sharded_by_companies:
  display(edge)

,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
4,adagio-catalyst,Research Scientist,4
32,objective-riesling,ASP.NET / C# / SQL 2005 / XML / XSD,32
33,objective-riesling,PHP / MySQL,32
40,objective-riesling,ASP / CRM / HRXML,40
...,...,...,...
1761624,dense-bell,Manager of Engineering,1761624
1761644,brave-hoops,"Vice President, Government Relations",1761644
1761648,brave-hoops,Legislative Counsel: U.S. House of Representat...,1761648
1761782,fixed-partition,Account Supervisor,146464


,_id,title,company_id
1,adagio-catalyst,Associate at Dept of Molecular Genetics,1
25,tart-acorn,Assistant Professor,25
29,objective-riesling,Ning,29
37,objective-riesling,MS Access,37
41,objective-riesling,ASP.NET / C# / SQL 2005 / IIS 6.0 / Clients Cu...,41
...,...,...,...
1761637,brave-hoops,Executive Director / Founder,1761637
1761640,brave-hoops,Director of Strategic Initiatives & Internatio...,1625009
1761641,brave-hoops,Task Force Member,1761641
1761645,brave-hoops,"Deputy Controller, Legislation",1761645


,_id,title,company_id
2,adagio-catalyst,Associate Research Scientist,2
26,tart-acorn,Technology Development Fellow,26
30,objective-riesling,.Net 4.0 / C# / VS 2010 / Silverlight 4.0 / Bl...,30
34,objective-riesling,VB 6 / SQL,34
42,objective-riesling,EDI setup,42
...,...,...,...
1761577,grouchy-flight,Account Management Intern,56538
1761578,grouchy-flight,Intern,1761578
1761638,brave-hoops,Chief Executive Officer,1761638
1761642,brave-hoops,Deputy Secretary for Information Technology,1761642


,_id,title,company_id
3,adagio-catalyst,Post Doctoral Research Fellow,3
31,objective-riesling,Owner,31
35,objective-riesling,ASP.NET / WSDL / JavaScript,35
36,objective-riesling,ASP.NET / C# / SQL 2005,35
38,objective-riesling,C# / MS Vista / Ant Profiler,35
...,...,...,...
1761623,dense-bell,Managing Partner,1761623
1761639,brave-hoops,Strategy Council Member,1761639
1761643,brave-hoops,Executive Management & Consulting Services,110295
1761647,brave-hoops,Counsel: U.S. House of Representatives,1761647


## Let's Do Some Computation!

### Single Join: People -> works_for

To do this, we need to be careful to join only between tables that are sharded by the join key (_id).

In [28]:
# sharded_persons is sharded by _id
# sharded_by_user is worked_for sharded by _id
for table in join_shards(sharded_persons, sharded_by_user, '_id', '_id'):
  display(table)

# Show the final results of the join
display(pd.concat(join_shards(sharded_persons, sharded_by_user, '_id', '_id')))

,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3
4,adagio-catalyst,Watt,Brunton,Research Scientist,4
...,...,...,...,...,...
63626,grouchy-flight,Ogilvy,Bullimore,Intern,1761578
63627,dense-bell,Macdougall,Barrymore,Principal,1761621
63628,dense-bell,Macdougall,Barrymore,Project Manager,913976
63629,dense-bell,Macdougall,Barrymore,Managing Partner,1761623


,_id,family_name,given_name,title,company_id
0,tart-acorn,Hannay,Passepartout,Assistant Professor,25
1,tart-acorn,Hannay,Passepartout,Technology Development Fellow,26
2,salty-section,Watt,Figaro,Founding Partner and Client Services Director,870
3,salty-section,Watt,Figaro,Managing Partner,871
4,salty-section,Watt,Figaro,Senior Account Director,872
...,...,...,...,...,...
63937,brave-hoops,Forsyth,Cadbury,Legislative Counsel: U.S. House of Representat...,1761648
63938,fixed-partition,Donald,Alfred,"Senior Vice President, Director",1761781
63939,fixed-partition,Donald,Alfred,Account Supervisor,146464
63940,fixed-partition,Donald,Alfred,Account Executive,1761783


,_id,family_name,given_name,title,company_id
0,afraid-collision,Hunter,Merriman,Zaakvoerster,1080
1,afraid-collision,Hunter,Merriman,HR,1081
2,afraid-collision,Hunter,Merriman,Branch Manager Kapellen at Adecco - Maritime/I...,1082
3,afraid-collision,Hunter,Merriman,Branch Manager,1082
4,afraid-collision,Hunter,Merriman,Branch Manager Schoten - Lier,1084
...,...,...,...,...,...
64050,late-cadet,Sutherland,Passepartout,Project Manager,81256
64051,late-cadet,Sutherland,Passepartout,IT Director,995714
64052,late-cadet,Sutherland,Passepartout,SR IS Business Analyst,890751
64053,late-cadet,Sutherland,Passepartout,Manager,13999


,_id,family_name,given_name,title,company_id
0,sensitive-estuary,Macneil,Merriman,Sales Manager - Security Team,979
1,sensitive-estuary,Macneil,Merriman,Information Security Consultant,979
2,sensitive-estuary,Macneil,Merriman,IT Security Sales Consultant,981
3,sensitive-estuary,Macneil,Merriman,Sales Manager,982
4,sensitive-estuary,Macneil,Merriman,Corporate Account Manager,983
...,...,...,...,...,...
65190,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179
65191,resultant-coach,Kincaid,Bertuccio,"Chairman, Board of Directors",1761458
65192,resultant-coach,Kincaid,Bertuccio,Associate,1761459
65193,resultant-coach,Kincaid,Bertuccio,Associate,1761460


,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3
4,adagio-catalyst,Watt,Brunton,Research Scientist,4
...,...,...,...,...,...
65190,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179
65191,resultant-coach,Kincaid,Bertuccio,"Chairman, Board of Directors",1761458
65192,resultant-coach,Kincaid,Bertuccio,Associate,1761459
65193,resultant-coach,Kincaid,Bertuccio,Associate,1761460


So, 256823 rows!

We can prove the above is correct by looking at the standard, unsharded result.

In [29]:
# Traditional Pandas query, check the count and results
persons_df.merge(worked_for, left_on='_id', right_on='_id')

,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3
4,adagio-catalyst,Watt,Brunton,Research Scientist,4
...,...,...,...,...,...
256818,brave-hoops,Forsyth,Cadbury,Legislative Counsel: U.S. House of Representat...,1761648
256819,fixed-partition,Donald,Alfred,"Senior Vice President, Director",1761781
256820,fixed-partition,Donald,Alfred,Account Supervisor,146464
256821,fixed-partition,Donald,Alfred,Account Executive,1761783


### What If We Don't Watch Our Sharding?

What if we don't align the shard columns and join keys? Let's see...

In [30]:
# sharded_persons is sharded by _id
# sharded_by_companies is worked_for sharded by the company

# Output the result of the join without proper sharding
pd.concat(join_shards(sharded_persons, sharded_by_companies, '_id', '_id'))

,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Research Scientist,4
2,objective-riesling,Carnegie,Passepartout,ASP.NET / C# / SQL 2005 / XML / XSD,32
3,objective-riesling,Carnegie,Passepartout,PHP / MySQL,32
4,objective-riesling,Carnegie,Passepartout,ASP / CRM / HRXML,40
...,...,...,...,...,...
16544,espressivo-media,Dundas,Brunton,"Senior Director, Product Development",1760983
16545,espressivo-media,Dundas,Brunton,Associate,20167
16546,natural-customer,Charteris,Passepartout,District Team Leader - Oklahoma,1761443
16547,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179


What happens if we *broadcast* one of the relations?

We would generally only do this with a small table.

In [31]:
pd.concat(join_shard_with_broadcast(sharded_persons, worked_for, '_id', '_id'))

,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3
4,adagio-catalyst,Watt,Brunton,Research Scientist,4
...,...,...,...,...,...
65190,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179
65191,resultant-coach,Kincaid,Bertuccio,"Chairman, Board of Directors",1761458
65192,resultant-coach,Kincaid,Bertuccio,Associate,1761459
65193,resultant-coach,Kincaid,Bertuccio,Associate,1761460


## Now Let's Do a Full Connection

OK, we saw one join.  If we want to traverse our network from people to companies, we need two joins.

Here's the Pandas query we want to do in a sharded model...

In [32]:
persons_df.merge(worked_for, left_on='_id', right_on='_id').merge(companies_df, left_on='company_id', right_on='company_id')

,_id,family_name,given_name,title,company_id,org
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1,Albert Einstein Medical Center
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2,Columbia University
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3,Memorial Sloan Kettering Cancer Center
4,adagio-catalyst,Watt,Brunton,Research Scientist,4,Sahlgrenska University Hospital
...,...,...,...,...,...,...
256818,brave-hoops,Forsyth,Cadbury,Legislative Counsel: U.S. House of Representat...,1761648,Congresswoman Cardiss Collins
256819,fixed-partition,Donald,Alfred,"Senior Vice President, Director",1761781,M Booth & Associates
256820,fixed-partition,Donald,Alfred,Account Supervisor,146464,Hill & Knowlton
256821,fixed-partition,Donald,Alfred,Account Executive,1761783,Richmond Public Relations


To join with `companies_df` we need to repartition our intermediate result on the company.

In [33]:
# This is the join from above
persons_work_for = join_shards(sharded_persons, sharded_by_user, '_id', '_id')

# persons_work_for is sharded by _id
# sharded_companies is sharded by company
# to join them, we need to find a common sharding key -- the company

# SO: we repartition persons_work_for by company
persons_companies = join_shards(repartition(persons_work_for, 'company_id'), sharded_companies, 'company_id', 'company_id')

pd.concat(
persons_companies
)

,_id,family_name,given_name,title,company_id,org
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson
1,adagio-catalyst,Watt,Brunton,Research Scientist,4,Sahlgrenska University Hospital
2,objective-riesling,Carnegie,Passepartout,ASP.NET / C# / SQL 2005 / XML / XSD,32,<Software Consulting Company>
3,objective-riesling,Carnegie,Passepartout,PHP / MySQL,32,<Software Consulting Company>
4,objective-riesling,Carnegie,Passepartout,ASP / CRM / HRXML,40,<Employment Agency>
...,...,...,...,...,...,...
65338,espressivo-media,Dundas,Brunton,"Senior Director, Product Development",1760983,Ticketmaster - CitySearch
65339,espressivo-media,Dundas,Brunton,Associate,20167,AT&T
65340,natural-customer,Charteris,Passepartout,District Team Leader - Oklahoma,1761443,Target Stores
65341,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179,Wells Fargo Securities


Again, this matches our Pandas query.

## Two Hops

Can we find employees who have a common employer?

Let's divide it into steps.

First, let's do a building block, namely the set of people, companies they work for, and the interconnecting `works` relationships:

```
people --> works_for --> company <-- works_for
[---- (persons_companies) -----]
```

In [34]:
# sharded_by_companies is the works_for edge, sharded by company
second_works_for = rename_shards(sharded_by_companies, {'_id': '_id2', 'role': 'role2'})

# We are currently sharded on company
person_works_company_works = \
    filter_shards(join_shards(persons_companies, second_works_for, 'company_id', 'company_id'),
           lambda row: row['_id'] != row['_id2']
    )
pd.concat(person_works_company_works)

,_id,family_name,given_name,title_x,company_id,org,_id2,title_y
1,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,paper-scarfing,GOLD Associate (Global Operations Leadership D...
2,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,paper-scarfing,Summer Intern
3,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,sharp-deck,Professional Sales Representative
4,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,stern-bus,Senior Consultant
31,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,fixed-triplet,Intern
...,...,...,...,...,...,...,...,...
1703227,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,overcast-jetsam,"OI DSG Engineer, GT3"
1703228,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,associative-restaurant,"Intern, Sales Compliance and Contract Management"
1703231,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179,Wells Fargo Securities,endothermic-grape,Sr. ETL/Data Architect
1703232,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179,Wells Fargo Securities,spatial-strategy,Quantitative Associate


Now we can join with another company of the

In [35]:
sharded_persons_2 = rename_shards(sharded_persons, {'_id': '_id2', 'given_name': 'given_name2', 'last_name': 'last_name2'})

result = join_shards(repartition(person_works_company_works, '_id2'), sharded_persons_2, '_id2', '_id2')
# Take the above and shard + join with sharded_persons
for table in result:
  display(table)

display(pd.concat(result))

,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,nondeterministic-fractal,VP Technical Services and CISO,Forsyth,Jeeves
1,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,dark-throw,Talent Development Manager (People),Macneil,Barrymore
2,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,dark-throw,Training Store Director,Macneil,Barrymore
3,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,joint-plot,Business Development Manager,Douglas,Bertuccio
4,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,tossed-queue,Solutions Consultant,Brodie,Figaro
...,...,...,...,...,...,...,...,...,...,...
1268927,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,noisy-elevation,Staff Supervisor,Robertson,Cadbury
1268928,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,tart-displacement,Sr. Member of Technical Staff,Haldane,Figaro
1268929,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,primordial-story,Manager Systems Design and Development,Sutherland,Passepartout
1268930,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,windy-parkway,Technical Director,Gordon,Bertuccio


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,andante-account,"Senior Product Manager, Social",Campbell,Merriman
1,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,undecidable-differential,Server Systems Engineer,Murray,Simonides
2,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,concrete-galley,Senior Project Manager IT (consultant),Fraser,Jenkins
3,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,giant-bounce,Student Associate,Hamilton,Simonides
4,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,grouchy-ranch,Manager of Quality Assurance - Juris,Fraser,Alfred
...,...,...,...,...,...,...,...,...,...,...
1326689,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,andante-stadium,Vice President - Sales,Macdougall,Bullimore
1326690,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,andante-stadium,Vice President - General Manager,Macdougall,Bullimore
1326691,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,crazy-trigger,Principal Engineer,Barclay,Cadbury
1326692,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,sunny-render,Support & Project Coodinator,Mair,Barkley


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,sharp-deck,Professional Sales Representative,Maclaren,Barrymore
1,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,stern-bus,Senior Consultant,Buchanan,Barkley
2,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,fixed-triplet,Intern,Malcolm,Figaro
3,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,inverse-balinese,Make-up Artist (PT),Oliphant,Bertuccio
4,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,one-helm,Category Manager Europe Skincare,Sterling,Leporello
...,...,...,...,...,...,...,...,...,...,...
1299889,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,congruent-app,Information System Senior Staff Member,Grierson,Simonides
1299890,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,upbeat-bridge,Senior Analyst,Hay,Belvedere
1299891,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,vertical-spot,Consultant,Montgomery,Merriman
1299892,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,sleek-puffin,"Sr .Vice President, Network Development & Rese...",Burnett,Alfred


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,paper-scarfing,GOLD Associate (Global Operations Leadership D...,Montgomery,Cadbury
1,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,paper-scarfing,Summer Intern,Montgomery,Cadbury
2,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,spicy-template,Senior Designer,Kennedy,Poole
3,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,joint-opportunity,Assistant Manager,Macneil,Cadbury
4,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,decidable-loveseat,Human Resources Executive,Macmillan,Brunton
...,...,...,...,...,...,...,...,...,...,...
1307399,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,stubborn-glass,Enterprise Application Developer,Chattan,Merriman
1307400,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,amber-dessert,Contractor,Macpherson,Alfred
1307401,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,overcast-jetsam,"OI DSG Engineer, GT3",Morr,Bullimore
1307402,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,associative-restaurant,"Intern, Sales Compliance and Contract Management",Elliott,Brunton


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,nondeterministic-fractal,VP Technical Services and CISO,Forsyth,Jeeves
1,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,dark-throw,Talent Development Manager (People),Macneil,Barrymore
2,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,dark-throw,Training Store Director,Macneil,Barrymore
3,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,joint-plot,Business Development Manager,Douglas,Bertuccio
4,massive-truss,Forbes,Jenkins,Research & Consulting Software Engineer,3512,LexisNexis,tossed-queue,Solutions Consultant,Brodie,Figaro
...,...,...,...,...,...,...,...,...,...,...
1307399,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,stubborn-glass,Enterprise Application Developer,Chattan,Merriman
1307400,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,amber-dessert,Contractor,Macpherson,Alfred
1307401,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,overcast-jetsam,"OI DSG Engineer, GT3",Morr,Bullimore
1307402,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,associative-restaurant,"Intern, Sales Compliance and Contract Management",Elliott,Brunton


## Exercise

Let's do this via a broadcast join.

In [36]:
# TODO
persons_2_df = persons_df.rename(columns={
    '_id': '_id2',
    'given_name': 'given_name2',
    'last_name': 'last_name2'
})
result = join_shard_with_broadcast(person_works_company_works,
    persons_2_df,
    '_id2',
    '_id2'
)

display(pd.concat(result))

,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,paper-scarfing,GOLD Associate (Global Operations Leadership D...,Montgomery,Cadbury
1,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,paper-scarfing,Summer Intern,Montgomery,Cadbury
2,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,sharp-deck,Professional Sales Representative,Maclaren,Barrymore
3,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson,stern-bus,Senior Consultant,Buchanan,Barkley
4,exact-novel,Morr,Barrymore,marketing and communication manager,1412,Sephora,fixed-triplet,Intern,Malcolm,Figaro
...,...,...,...,...,...,...,...,...,...,...
1786613,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,overcast-jetsam,"OI DSG Engineer, GT3",Morr,Bullimore
1786614,espressivo-media,Dundas,Brunton,Associate,20167,AT&T,associative-restaurant,"Intern, Sales Compliance and Contract Management",Elliott,Brunton
1786615,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179,Wells Fargo Securities,endothermic-grape,Sr. ETL/Data Architect,Forsyth,Simonides
1786616,resultant-coach,Kincaid,Bertuccio,"Director - Head of CDO Management, Asset Backe...",289179,Wells Fargo Securities,spatial-strategy,Quantitative Associate,Riddell,Belvedere


In [37]:
grader.grade('sharded_net_result', len(pd.concat(result)))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
